# Image Scaling with Machine Learning

## Parameters to generate a set of images for train-test-validate
    * Over all parameters
        * number of image sets
        * sizes dict {name1: [h, w], name2: [h, w],...}
        * run-limits {escape_boundry: diagonal_multiplier, max_iterations: int}
        * coloring scheme - perhaps a function
        * output file format
        * ouput file naming format
    * File system
        * output directory {train: path, test: path, validate: path}
        * run-log location
    * Stocastic Choices:
        * hashed dictionary - to reject duplicate images
        * equation
            * parameters
        * rotation 
        * center point 
        * frame scale

## Source structure
    * main - opens run_parameters file - passes to module function
    * selection and tracking module
    * starting plain complex
    * equations module
    * iteration module
    * matrix to image module
    
****
[GitHub scalygraphic](https://github.com/dlanier/scalygraphic/) <br>
### Background Code:
[Add: random plane on domain](https://github.com/dlanier/scalygraphic/blob/master/scalygraphic/zplain.py) <br>

#### Import & Expand *functions_demo.py*
[FMF functions_demo](https://github.com/dlanier/FlyingMachineFractal/blob/master/pyreimpic/functions_demo_01.py) <br>
#### Import & Trim itergators.py
[FMF itergators](https://github.com/dlanier/FlyingMachineFractal/blob/master/src/itergataters.py) <br>
#### Rewrite HSV coloring
[FMF graphic_utility.py](https://github.com/dlanier/FlyingMachineFractal/blob/master/src/graphic_utility.py) <br>
[FMF numcolorpy.py](https://github.com/dlanier/FlyingMachineFractal/blob/master/src/numcolorpy.py) <br>
#### Reslove plain functions
[FMF z_plane.py](https://github.com/dlanier/FlyingMachineFractal/blob/master/src/z_plane.py) <br>
****
    
### Inception: Random Selection, Tracking and Generation - called from *main()*

In [ ]:
p_all = {'number_of_image_sets': 10, 
         'sizes_dict': {'small': [128, 128], 'large': [1024, 1024]}, 
         'run_limits': {'esc_dist': 12, 'max_iterations': 64}, 
         'color': 'rgb', 
         'file_format': 'jpg', 
         'output_directory': './', 
         'run_directory': '../'}


In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# Clickety Clack:
## production_parameters: *output_dir_name, image_sizes_dict, number_of_sets*
### get a random eq & p with a random domain
#### check uniquness
### get a random color map
## Iterate: run, write, report

In [2]:
import os
import sys
import hashlib
import numpy as np

sys.path.insert(0, '../src/')
from eq_iter import get_rand_eq_p_set
import zplain as zp

## get a random function and parameter set

In [3]:
n_trys = 3
for k in range(n_trys):
    fcn_name, eq, p = get_rand_eq_p_set()
    print('\n\t', fcn_name)
    try:
        Z = 0.0+0.0j
        print(Z)
        print(eq(Z,p))
        Z = 1.0+1.0j
        print(Z)
        print(eq(Z,p))
    except:
        print('Crash crash ')
        break
        pass


	 decPwrAFx
0j
inf
(1+1j)
(0.16528862469723804+1.9184157694253376j)

	 RoyalZ
0j
inf
(1+1j)
(0.2923148390749084-0.3868784485455075j)

	 ItchicuPpwrF
0j
0j
(1+1j)
(-25.981292978230876-33.45441127947091j)


## get a random complex domain

In [4]:
# from zplain import get_complex_frame, complex_frame_dict_to_string, get_frame_from_dict
# import zplain as zp

In [5]:
# CP_magnitude_limits =   {'min': 0, 'max': 10}
# ZM_limits =             {'min': np.finfo(float).eps, 'max': 1}
# theta_limits =          {'min': 0, 'max':2 * np.pi}

def get_random_domain(h, w, bounds_dict=None):
    if bounds_dict is None:
        CP_magnitude_limits =   {'min': 0, 'max': 7}
        ZM_limits =             {'min': np.finfo(float).eps, 'max': 2}
        theta_limits =          {'min': 0, 'max':2 * np.pi}
    else:
        CP_magnitude_limits =   bounds_dict['CP_magnitude_limits']
        ZM_limits =             bounds_dict['ZM_limits']
        theta_limits =          bounds_dict['theta_limits']

    def_dict = {'n_rows': h, 'n_cols': w}
    r = np.random.uniform(low=0.0, high=2*np.pi) * 0.0+1.0j
    m = np.random.uniform(low=CP_magnitude_limits['min'], high=CP_magnitude_limits['max'])
    def_dict['center_point'] = m*np.exp(r)
    def_dict['zoom'] = np.random.uniform(low=ZM_limits['min'], high=ZM_limits['max'])
    def_dict['theta'] = np.random.uniform(low=theta_limits['min'], high=theta_limits['max'])
    
    return def_dict

N_DEC=4

domain_dict = get_random_domain(h=7, w=5)
f = zp.get_frame_from_dict(domain_dict)
s = zp.complex_frame_dict_to_string(f,N_DEC)
print(s)

upper_left:3.9692+3.2649j	top_center:3.2687+2.5489j	upper_right:2.5682+1.8330j
left_center:2.9669+4.2456j	center_point:4.5327+7.0593j	right_center:1.5659+2.8137j
bottom_left:1.9645+5.2263j	bottom_center:1.2640+4.5103j	bottom_right:0.5635+3.7944j



In [6]:
def sha256sum(s):
    """
    """
    h  = hashlib.sha256()
    h.update(bytes(s, 'ascii'))
    
    return h.hexdigest()

def hash_parameters(domain_dict, fcn_name, p):
    """ Usage: hash_key = hash_parameters(domain_dict, fcn_name, p)
    """
    N_DEC = 15
    f = zp.get_frame_from_dict(domain_dict)
    s = zp.complex_frame_dict_to_string(f, N_DEC) + '\n' + fcn_name
    if isinstance(p, list):
        p_str = ''
        for p_n in p:
            p_str += zp.complex_to_string(p_n, N_DEC)
    else:
        p_str = zp.complex_to_string(p, N_DEC)
    
    s += p_str
    
    return sha256sum(s)




fcn_name, eq, p = get_rand_eq_p_set()
domain_dict = get_random_domain(h=7, w=5)

hash_key = hash_parameters(domain_dict, fcn_name, p)
print(hash_key)

52874b655dc5aa4f247c6795cb58baaeda02c400aa0cd64293686b449ddbffd7


In [7]:
z = 0.0+1.0j
r = 0.0
print(isinstance(z, complex), isinstance(r, complex))
print(zp.complex_to_string(z),zp.complex_to_string(r))

True False
0.000+1.000j 0.000+0.000j


In [9]:
import inspect
import deg_0_ddeq

all_functions = inspect.getmembers(deg_0_ddeq, inspect.isfunction)


In [12]:
type(all_functions), type(all_functions[0])

(list, tuple)

In [13]:
af_dict = dict(all_functions)

In [14]:
for k, v in af_dict.items():
    print(k, type(v))

IslaLace <class 'function'>
ItchicuPpwrF <class 'function'>
Nautuliz <class 'function'>
RoyalZ <class 'function'>
T_Spake_Z <class 'function'>
decPwrAFx <class 'function'>
dreadSkull <class 'function'>
starfish_ish <class 'function'>
starfish_ish_II <class 'function'>
